In [1]:
function matrixize(lines)
    get_row(line) = split(line, "") .|> x -> parse(Int, x)
    rows = get_row.(lines)
    return hcat(rows...)'
end

matrixize (generic function with 1 method)

In [75]:
lines = readlines("input.txt")
grid = matrixize(lines)

100×100 adjoint(::Matrix{Int64}) with eltype Int64:
 4  3  9  5  1  9  4  5  7  5  9  2  9  …  7  2  5  1  3  2  1  9  3  6  6  2
 1  7  1  5  7  1  9  2  2  8  7  5  9     8  4  2  7  9  2  9  7  9  8  3  6
 5  6  4  1  9  5  7  9  4  7  3  9  9     9  3  5  9  2  6  9  8  9  9  6  5
 2  5  8  5  4  9  7  8  5  1  3  1  8     8  6  1  7  3  2  9  4  9  9  4  2
 6  3  2  6  1  2  1  2  9  3  6  2  9     9  1  7  8  9  6  9  7  1  9  1  9
 1  1  9  7  8  9  1  1  9  3  1  6  3  …  7  9  6  8  3  3  3  2  8  7  9  7
 9  9  7  1  9  4  1  1  7  6  2  3  6     7  5  9  8  9  7  9  4  6  8  8  3
 8  9  9  9  2  9  4  4  2  2  7  1  1     9  7  2  3  9  9  2  8  9  8  9  6
 3  3  4  9  2  9  1  4  9  3  8  8  7     7  9  7  1  8  3  6  5  9  1  5  7
 7  1  8  9  4  3  9  8  3  5  6  1  6     5  8  8  3  4  2  7  1  7  1  7  4
 9  9  1  9  6  8  7  9  3  1  9  9  1  …  8  6  1  1  9  2  2  2  7  8  1  3
 8  5  5  6  7  5  2  1  1  9  9  4  8     4  9  9  8  8  1  7  6  9  8  7  7
 1  9  7  9 

In [76]:
function is_in_bounds(point, grid_size)
    return (1 ≤ point[1] ≤ grid_size[1]) && (1 ≤ point[2] ≤ grid_size[2])
end

function get_neighbours(point, grid_size)
    neighbours = [(point[1] + deltas[1], point[2] + deltas[2]) for deltas in [(-1, 0), (1, 0), (0, -1), (0, 1)]]
    return filter(x -> is_in_bounds(x, grid_size), neighbours)
end

get_neighbours (generic function with 1 method)

In [77]:
using Pkg
Pkg.add("DataStructures")

import DataStructures: DefaultDict, BinaryHeap

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`


In [78]:
Coordinate = Tuple{Int, Int}

heuri(point, end_point) = abs(end_point[1] - point[1]) + abs(end_point[2] - point[2])

lowest_f_score(points, f_scores) = BinaryHeap(Base.By(first), [(f_scores[p], p) for p in points]) |> first |> last

function reconstruct_path(came_from, current)
    total_path = [current]
    while current in keys(came_from)
        current = came_from[current]
        pushfirst!(total_path, current)
    end
    return total_path
end

# courtesy of https://en.wikipedia.org/wiki/A%2a_search_algorithm
function a_star(start, goal, grid)
    grid_size = size(grid)
    open_set = Set([start])
    came_from = Dict{Coordinate, Coordinate}()
    
    g_scores = DefaultDict{Coordinate, Float64}(Inf)
    g_scores[start] = 0
    
    f_scores = DefaultDict{Coordinate, Float64}(Inf)
    f_scores[start] = heuri(start, goal)
    
    while length(open_set) > 0
        current = lowest_f_score(open_set, f_scores)
        
        if current == goal
            # return reconstruct_path(came_from, current)
            return g_scores[current]
        end
        
        setdiff!(open_set, [current])

        for neighbor in get_neighbours(current, grid_size)
            tentative_g_score = g_scores[current] + grid[CartesianIndex(neighbor)]
            if tentative_g_score < g_scores[neighbor]
                came_from[neighbor] = current
                g_scores[neighbor] = tentative_g_score
                f_scores[neighbor] = tentative_g_score + heuri(neighbor, goal)
                if !(neighbor in open_set)
                    push!(open_set, neighbor)
                end
            end
        end
    end
    return nothing
end

a_star (generic function with 1 method)

In [79]:
a_star((1,1), size(grid), grid)

619.0

In [80]:
add(n, grid) = (grid .+ (n-1)) .% 9 .+1

add (generic function with 1 method)

In [81]:
function expand(n, grid)
    blocks = [add(i, grid) for i in 0:n-1]
    macro_cols = [vcat([add(i, b) for i in 0:n-1]...) for b in blocks]
    return hcat(macro_cols...)
end

expand (generic function with 1 method)

In [82]:
big_grid = expand(5, grid)

500×500 Matrix{Int64}:
 4  3  9  5  1  9  4  5  7  5  9  2  9  …  2  6  9  5  7  6  5  4  7  1  1  6
 1  7  1  5  7  1  9  2  2  8  7  5  9     3  8  6  2  4  6  4  2  4  3  7  1
 5  6  4  1  9  5  7  9  4  7  3  9  9     4  7  9  4  6  1  4  3  4  4  1  9
 2  5  8  5  4  9  7  8  5  1  3  1  8     3  1  5  2  7  6  4  8  4  4  8  6
 6  3  2  6  1  2  1  2  9  3  6  2  9     4  5  2  3  4  1  4  2  5  4  5  4
 1  1  9  7  8  9  1  1  9  3  1  6  3  …  2  4  1  3  7  7  7  6  3  2  4  2
 9  9  7  1  9  4  1  1  7  6  2  3  6     2  9  4  3  4  2  4  8  1  3  3  7
 8  9  9  9  2  9  4  4  2  2  7  1  1     4  2  6  7  4  4  6  3  4  3  4  1
 3  3  4  9  2  9  1  4  9  3  8  8  7     2  4  2  5  3  7  1  9  4  5  9  2
 7  1  8  9  4  3  9  8  3  5  6  1  6     9  3  3  7  8  6  2  5  2  5  2  8
 9  9  1  9  6  8  7  9  3  1  9  9  1  …  3  1  5  5  4  6  6  6  2  3  5  7
 8  5  5  6  7  5  2  1  1  9  9  4  8     8  4  4  3  3  5  2  1  4  3  2  2
 1  9  7  9  9  4  7  8  1  9  6  9  8   

In [83]:
a_star((1,1), size(big_grid), big_grid)

2922.0